In [1]:
import numpy as np
import pandas as pd

In [2]:
input_csv = '/Volumes/vol/work/Github/playground/ggmparser/GGM_trio_UBN1.csv'
df = pd.read_csv(input_csv, header=0, dtype=str)


In [8]:
df.columns

Index(['No./242', 'Sample', 'Gene', 'Transcript', 'Impact', 'Effect',
       'Distance', 'Amino acid change2', 'SIFT', 'PolyPhen-2',
       'MutationTaster', 'CADD', 'alleleID', 'Chr', 'Position', 'Ref', 'Alt',
       'GGM(AC/AN)', 'ToMMo3.5K(AC)', 'ToMMo3.5K(AN)', 'ToMMo3.5K(AF)',
       'JPNCTL(SC)', 'JPNCTL(SN)', 'ExAC(AC)', 'ExAC(AF)', 'gnomAD(AC)',
       'gnomAD(AN)', 'gnomAD(AF)', 'Family', 'Vtype', 'Disease', 'ID(pro)',
       'AS(pro)', 'GT(pro)', 'GQ:DP:AD(pro)', 'ID(pat)', 'AS(pat)', 'GT(pat)',
       'GQ:DP:AD(pat)', 'ID(mat)', 'AS(mat)', 'GT(mat)', 'GQ:DP:AD(mat)',
       'Analysis status', 'Identified gene', 'Variant description', 'GQ(pro)',
       'DP(pro)', 'AD(pro)', 'GQ(pat)', 'DP(pat)', 'AD(pat)', 'GQ(mat)',
       'DP(mat)', 'AD(mat)'],
      dtype='object')

In [6]:
def split_qc_col(df: pd.DataFrame) -> pd.DataFrame:
    for rel in ['pro', 'pat', 'mat']:
        df = pd.concat(
            [df, df[f'GQ:DP:AD({rel})'].str.split(':', expand=True)], axis=1)
        for i in range(3):
            df[i] = df[i].replace('.', np.nan)
            df[i] = df[i].replace('-', np.nan)
        df = df.astype({0: float, 1: float, 2: float})
        df.rename(columns={0: f'GQ({rel})', 1: f'DP({rel})', 2: f'AD({rel})'}, 
                inplace=True)
    return df

def maf_filter(df: pd.DataFrame, th: float) -> pd.DataFrame:
    df.loc[((df['ToMMo3.5K(AF)'] < th)  
            & (df[('ToMMo3.5K(AF)')].notna())), 
           'ToMMo3.5K(AF)'] = np.nan
    
    
    return df

In [9]:
df['ToMMo3.5K(AF)'].value_counts()

ToMMo3.5K(AF)
-         126
0.0001     68
0.0004      7
0.001       7
0.0007      6
0.0034      5
0.0019      5
0.0009      4
0.0003      3
0.0006      2
0.0008      2
0.0022      2
0.0027      2
0.0017      1
0.0002      1
0.0021      1
Name: count, dtype: int64